# Deep Learning Classification of Anomaly Peaks with 1D resnet
Using normal train/test split

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn import preprocessing
from resnet import ResNet1D
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

sys.path.insert(1, "../")

from datasets import fdomDataset


In [ ]:
# Hyperparams
WINDOW_SIZE = 15 # the size of each data segment
TEST_SIZE = 0.10
SEED = 42
BATCH_SIZE = 32

In [ ]:
# Paths to data files
fdom_raw_data = (
    "../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv"
)
stage_raw_data = "../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv"
turb_raw_data = (
    "../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv"
)

fdom_labeled = "../Data/labeled_data/ground_truths/fDOM/fDOM_all_julian_0k-300k.csv"


In [ ]:
# get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


## Create dataset and dataloaders

In [ ]:
# create dataset
classes = ["NAP", "FSK", "FPT", "PLP", "PP", "SKP"]
le = preprocessing.LabelEncoder()

targets = le.fit_transform(classes)

dataset = fdomDataset(
    le,
    fdom_raw_data,
    stage_raw_data,
    turb_raw_data,
    fdom_labeled,
    window_size=WINDOW_SIZE
)


### Split into training/testing
This should not be the final iteration, this is just to get initial results.

In [ ]:
# split data into training / testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# create dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


In [ ]:
# init model
model = ResNet1D(
    in_channels=WINDOW_SIZE * 2 + 1,
    base_filters=64,
    kernel_size=16,
    stride=2,
    n_block=48,
    groups=1,  # check this
    n_classes=len(classes),
    downsample_gap=6,
    increasefilter_gap=12,
    verbose=False,
).to(device)

model = model.float()


## Init loss and optimizer

In [ ]:
# Optimizer/criterion
optimizer = optim.Adam(model.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss().to(device)
all_loss = []


## Train Model

In [ ]:
prog_bar = tqdm(trainloader, desc='Training', leave=False)
for i, batch in enumerate(prog_bar):
    x = batch[0].to(device)

    # squeeze y to flatten predictions into 1d tensor
    y = batch[1].squeeze().to(device)

    pred = model(x.float())

    loss = criterion(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    all_loss.append(loss.item())


## Test Model

In [ ]:
# Test model
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

prog_bar = tqdm(testloader, desc='Testing', leave=False)
with torch.no_grad():
    for i, batch in enumerate(prog_bar):
        x = batch[0].to(device)

        y = batch[1].squeeze().to(device)

        outs = model(x.float())

        _, preds = torch.max(outs, 1)

        for label, prediction in zip(y, preds):
            # convert label and prediction to current vals
            label = le.inverse_transform([label])[0]
            prediction = le.inverse_transform([prediction])[0]

            if label == prediction:
                correct_pred[label] += 1 # this may not work
            total_pred[label] += 1

for classname, correct_count in correct_pred.items():
    # because of unbalanced data, we need to not print out any classes that didn't have any labels
    if total_pred[classname] != 0:
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
    else:
        accuracy = 0.0
        print(f'CLASS NOT IN TEST BATCH: {classname:5s}')


## Display metrics

In [ ]:
# TODO: implement displaying metrics